# Token Flows

The idea of this notebook is to test some functionality for determining transaction classification

We want to classify a transaction into one of several categories (and more):

- trade
- add / remove liquidity
- lend / borrow / withdraw / repay
- claim rewards / other income
- simple transfer (internal or external)
- others

Determining transaction type informs tax status for a given transaction. 

Knowing the number of tokens going in and out from a wallet in a given transaction is useful information for making this classification.

For example: 

Typically adding deposit to an LP involves a move of 3 tokens: 2 tokens out (into the pool) and one token in (the LP token).
A 1-to-1 token in-out might indicate a trade / swap.
A 1 for 0 out might indicate a bridge transfer, a OTC trade, or a loan repayment (ignoring debt tokens for now)

While this approach is naive if taken without review, it is useful for filtering and viewing transactions.

This notebook includes some functions for making this process easy, building off of previous work elsewhere in this project, and then includes some code for additional filtering exercises on transaction data.

In [1]:
from src.transactions import *
from src.utils import *

import pandas as pd

In [2]:
# make transfers into a dictionary from a pandas dataframe
# structure is {hash: [array of transfers]}
def make_transfer_dict(_transfers=all_transfers) -> dict:
    transfers_dict = {}
    
    for _, transfer in _transfers.iterrows():
        # first time this hash has been seen
        if transfer.hash not in transfers_dict:
            transfers_dict[transfer.hash] = [transfer]
        # else concat the transfer to the existing value
        else:
            transfers_dict[transfer.hash] += [transfer]
    
    return transfers_dict


# count the number of token inflows for a given transactions
def count_inflows(_transfers, _wallets=WALLET_LIST) -> int:
    n_inflows = 0
    tokens_seen = []
    
    for transfer in _transfers:
        if transfer.transferTo in _wallets and transfer.tokenSymbol not in tokens_seen:
            n_inflows += 1
            tokens_seen += [transfer.tokenSymbol]
            
    return n_inflows


# count number of token outflows for a given transaction
def count_outflows(_transfers, _wallets=WALLET_LIST) -> int:
    n_outflows = 0
    tokens_seen = []
    
    for transfer in _transfers:
        if transfer.transferFrom in _wallets and transfer.tokenSymbol not in tokens_seen:
            n_outflows += 1
            tokens_seen += [transfer.tokenSymbol]
            
    return n_outflows


def get_m_for_n_trades(n_out=1, n_in=1, _transfers=all_transfers, _wallets=WALLET_LIST) -> dict:
    # a dictionary structure makes it much easier to traverse all hashes
    transfers_dict = make_transfer_dict(_transfers)
    hashes = []
    nton_transfers = []
    
    for hash_, transfers in transfers_dict.items():
        # make sure n_in flows are in and n_out flows are out
        if not count_inflows(transfers, _wallets) == n_in or not count_outflows(transfers, _wallets) == n_out:
            continue
            
        hashes += [hash_]
        
    # return dictionary items for these hashes
    return {hash_:transfers_dict[hash_] for hash_ in hashes} 


In [3]:
one_for_one = get_m_for_n_trades(1, 1, all_transfers, WALLET_LIST)
two_for_one = get_m_for_n_trades(2, 1, all_transfers, WALLET_LIST)


# some tests for one_for_one
for value in one_for_one.values():
    assert count_outflows(value) == 1
    assert count_inflows(value) == 1
    
assert len(set(one_for_one)) == len(one_for_one)

# some tests for two_for_one
for value in two_for_one.values():
    assert count_outflows(value) == 2
    assert count_inflows(value) == 1
    
assert len(set(two_for_one)) == len(two_for_one)

In [4]:
potential_trades = get_m_for_n_trades(1, 1)
potential_liquidity_adds = get_m_for_n_trades(2, 1)
potential_liquidity_withdraws = get_m_for_n_trades(1, 2)

In [5]:
six_one = get_m_for_n_trades(6,1)
print(len(six_one))

0


In [6]:
# get all trades of m for n tokens from m, n: 0..9
trade_sizes = {}
for i in range(10):
    trade_sizes[i] = {}
    print(i)
    for j in range(10):
        trade_sizes[i][j] = get_m_for_n_trades(i, j)

0
1
2
3
4
5
6
7
8
9


In [7]:
total = 0
for i in range(10):
    for j in range(10):
        n = len(trade_sizes[i][j])
        total += n
        if n != 0:
            print(f"{i}, {j}: {n}")

0, 1: 1628
0, 2: 503
0, 3: 106
0, 4: 9
0, 5: 11
0, 6: 18
0, 7: 8
1, 0: 625
1, 1: 2585
1, 2: 119
1, 3: 15
1, 4: 1
1, 5: 1
2, 0: 53
2, 1: 131
2, 2: 18
2, 3: 2
3, 1: 27
3, 3: 34
3, 4: 1
3, 6: 1
4, 1: 2
4, 4: 5
4, 5: 7
4, 7: 28
5, 1: 3
5, 6: 1
5, 7: 29
6, 7: 60
7, 1: 1


In [8]:
transfers_dict = make_transfer_dict(all_transfers)
print(total) # number of transactions with token sizes < 10
print(len(transfers_dict)) # total number of transactions

len(all_transfers.hash.unique()) # confirming unique hashes is same as size of transfers dictionary

6032
6035


6035

In [16]:
hashes = []
for i in range(10):
    for j in range(10):
        hashes += list(trade_sizes[i][j].keys())
        
count = 0 
for hash in all_transfers.hash.unique().tolist():
    if hash not in hashes:
        # found the three other transfers - all votium.app claims
        count += 1

print(count)

3


In [10]:
# find approve transactions
normal_transactions['action'] = [str(name).split('(')[0] for name in normal_transactions.functionName]

h = normal_transactions[normal_transactions.action == 'approve'].hash.iloc[0]

In [13]:
# calc gas fees
normal_transactions['txFee'] = normal_transactions.gasPrice * normal_transactions.gasUsed / 10 ** 18

# sort transactions by whether or not they are from us
not_from_us = normal_transactions[~normal_transactions['from'].isin(WALLET_LIST)]
from_us = normal_transactions[normal_transactions['from'].isin(WALLET_LIST)]

In [14]:
# total gas paid on mainnet by our wallets
print("Gas Paid by Chain")
print(f"mainnet:   {sum(from_us[from_us.chain == 'mainnet'].txFee)} ETH")
print(f"polygon:   {sum(from_us[from_us.chain == 'polygon'].txFee)} MATIC")
print(f"avalanche: {sum(from_us[from_us.chain == 'avalanche'].txFee)} AVAX")
print(f"arbitrum:  {sum(from_us[from_us.chain == 'arbitrum'].txFee)} ETH")
print(f"fantom:  {sum(from_us[from_us.chain == 'fantom'].txFee)} FTM")
print(f"binance:  {sum(from_us[from_us.chain == 'binance'].txFee)} BNB")

Gas Paid by Chain
mainnet:   28.271913866070676 ETH
polygon:   9.319300786971601 MATIC
avalanche: 7.785314829772161 AVAX
arbitrum:  0.24491394756031495 ETH
fantom:  147.08534929542944 FTM
binance:  0.000315 BNB
